<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/black_scholes_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Black-Scholes** --- Calculation price, greeks and implied volatility

*Reconciliation reference:*

*https://goodcalculators.com/black-scholes-calculator/*

*https://www.math.drexel.edu/~pg/fin/VanillaCalculator.html*

In [0]:
# Import packages
import numpy as np
from scipy.stats import norm

Black-Scholes analytical price

In [0]:
# Black-Scholes analytical function

def black_scholes_analytical(s_0, k, T, r, q, sigma, option_type):
  d1 = (np.log(s_0/k) + (r-q+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
  d2 = (np.log(s_0/k) + (r-q-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
  if option_type == 'call':
    val = np.exp(-q*T)*s_0*norm.cdf(d1,0,1) - k*np.exp(-r*T)*norm.cdf(d2,0,1)
  elif option_type == 'put':
    val = k*np.exp(-r*T)*norm.cdf(-d2,0,1) - s_0*np.exp(-q*T)*norm.cdf(-d1,0,1)
  return val

Black-Scholes class object for prices and greeks

In [0]:
# class objecct to price and determine greeks

class bs_object(object):
  """Black-Scholes model to provide present value and greeks for given inputs"""

  def __init__(self, s_0, k, T, r, q, sigma, option_type):
    self.s_0 = s_0
    self.k = k
    self.T = T
    self.r = r
    self.q = q
    self.sigma = sigma 
    self.option_type = option_type
    self.d1 = (np.log(self.s_0/self.k) + (self.r-self.q+0.5*self.sigma**2)*self.T) / (self.sigma*np.sqrt(self.T))
    self.d2 = (np.log(self.s_0/self.k) + (self.r-self.q-0.5*self.sigma**2)*self.T) / (self.sigma*np.sqrt(self.T))

  def val(self):
    if self.option_type == 'call':
      val = np.exp(-self.q*self.T)*self.s_0*norm.cdf(self.d1,0,1) - self.k*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1)
    elif self.option_type == 'put':
      val = self.k*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1) - self.s_0 * np.exp(-self.q*self.T)*norm.cdf(-self.d1,0,1)
    return val

  def delta(self):
    if self.option_type == 'call':
      delta = np.exp(-self.q*self.T)*norm.cdf(self.d1,0,1)
    elif self.option_type == 'put':
      delta = -np.exp(-self.q*self.T)*norm.cdf(-self.d1,0,1)
    return delta 

  def vega(self):
    vega = self.s_0*np.exp(-self.q*self.T)*np.sqrt(self.T)*norm.pdf(self.d1)
    return vega / 100

  def theta(self):
    if self.option_type == 'call':
      theta = (-np.exp(-self.q*self.T)*self.s_0*norm.pdf(self.d1)*self.sigma/(2*np.sqrt(self.T)) 
               - self.r*self.k*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1)
               + self.q*self.s_0*np.exp(-self.q*self.T)*norm.cdf(self.d1,0,1) 
               )
    elif self.option_type == 'put':
      theta = (-np.exp(-self.q*self.T)*self.s_0*norm.pdf(-self.d1)*self.sigma/(2*np.sqrt(self.T))
               + self.r*self.k*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1) 
               - self.q*self.s_0*np.exp(-self.q*self.T)*norm.cdf(-self.d1,0,1) 
               )
    return theta

  def rho(self):
    if self.option_type == 'call':
      rho = self.k*self.T*np.exp(-self.r*self.T)*norm.cdf(self.d2,0,1)
    elif self.option_type == 'put':
      rho = -self.k*self.T*np.exp(-self.r*self.T)*norm.cdf(-self.d2,0,1)
    return rho / 100   

  def gamma(self):
    gamma = np.exp(-self.q*self.T)*norm.pdf(self.d1) / (self.s_0*self.sigma*np.sqrt(self.T))
    return gamma

In [0]:
# Review class
help(bs_object)

Help on class bs_object in module __main__:

class bs_object(builtins.object)
 |  Black-Scholes model to provide present value and greeks for given inputs
 |  
 |  Methods defined here:
 |  
 |  __init__(self, s_0, k, T, r, q, sigma, option_type)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  delta(self)
 |  
 |  gamma(self)
 |  
 |  rho(self)
 |  
 |  theta(self)
 |  
 |  val(self)
 |  
 |  vega(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



Run Black-Scholes analytical pricer and class object

In [0]:
# Calculate black_scholes_analytical and bs_object

# Inputs
s_0_val = 102
k_val = 105
r_val = 0.05
q_val = 0.02
sigma_val = 0.15
T_val =  2
option_type_val = 'call'    # or 'put'
decimals = 3

# input collation for bs_object
trade_1 = bs_object(s_0_val, k_val, T_val, 
               r_val, q_val, sigma_val, option_type_val)

print('black_scholes_analytical=', round(
    black_scholes_analytical(s_0_val, k_val, T_val, r_val, q_val, 
                             sigma_val, option_type_val), decimals), '\n')

print('Results from bs_object:')
print('d1=', round(trade_1.d1, decimals))
print('d2=', round(trade_1.d2, decimals)) 
print('Val=', round(trade_1.val(), decimals)) 
print('Delta=', round(trade_1.delta(), decimals)) 
print('Vega=', round(trade_1.vega(), decimals)) 
print('Theta=', round(trade_1.theta(), decimals)) 
print('Rho=', round(trade_1.rho(), decimals)) 
print('Gamma=', round(trade_1.gamma(), decimals))

black_scholes_analytical= 9.735 

Results from bs_object:
d1= 0.252
d2= 0.04
Val= 9.735
Delta= 0.576
Vega= 0.536
Theta= -3.285
Rho= 0.98
Gamma= 0.017


Calculate implied volatility from option prices: 
i) Naive brute force approach
ii) Newton-Raphson

---



In [0]:
# Calculate implied volatility from option price
# Method is naive using brute force and finding nearest 
# For better results use Newton-Raphson 

def imp_vol(pv, s_0, k, r, q, T, option_type, epsilon, max_sigma, inc_sigma):
  sigma_array = np.linspace(inc_sigma_val,max_sigma_val, int(max_sigma_val/inc_sigma_val))
  pv_iter_array = []
  abs_diff_array = []
  for sigma in sigma_array:
    pv_iter = black_scholes_analytical(s_0, k, T, r, q, sigma, option_type)
    pv_iter_array.append(pv_iter)
    abs_diff = abs(pv - pv_iter)
    abs_diff_array.append(abs_diff)
  where_vol_diff_min = np.where(abs_diff_array == np.amin(abs_diff_array))[0][0]
  return sigma_array[where_vol_diff_min]

In [0]:
# Run imp_vol calculations

pv_val = 9.7
s_0_val = 102
k_val = 105
r_val = 0.05
q_val = 0.02
T_val =  2
option_type_val = 'call'    # or 'put'
epsilon_val = 2
max_sigma_val = 1
inc_sigma_val = 0.001
decimals = 4

round(imp_vol(pv_val, s_0_val, k_val, r_val, q_val, T_val, option_type_val, epsilon_val, max_sigma_val, inc_sigma_val), decimals)

0.149

In [0]:
# Estimate implied volatility using Newton-Raphson
def newtonRaphson( pv_target, s_0, k, T, r, q, sigma, option_type, accuracy ): 
    h = ( (bs_object(s_0, k, T, r, q, sigma, option_type).val() - pv_target) 
    / bs_object(s_0, k, T, r, q, sigma, option_type).vega() )
    while abs(h) >= accuracy:
      h = ( (bs_object(s_0, k, T, r, q, sigma, option_type).val() - pv_target) 
      / bs_object(s_0, k, T, r, q, sigma, option_type).vega() )
      sigma = sigma - h / 100
    return sigma 

In [0]:
# Newton-Raphson to estimate implied volatility

# Inputs
pv_target_val = 12
s_0_val = 102
k_val = 105
r_val = 0.05
q_val = 0.02
sigma_val = 0.05    # estimate implied volatility to begin
T_val =  2
option_type_val = 'call'    # or 'put'
accuracy_val = 0.01
decimals = 5

# Estimate implied vol
imp_vol_est = newtonRaphson(pv_target_val, s_0_val, k_val, T_val, r_val, q_val, sigma_val, option_type_val, accuracy_val)

# Test
pv_for_imp_vol_est = bs_object(s_0_val, k_val, T_val, r_val, q_val, imp_vol_est, option_type_val).val()
print('Estimated implied volatility for pv_target_val=', pv_target_val, 'is', round(imp_vol_est, decimals))
print('Compare with BS price for implied vol of =', round(imp_vol_est, decimals), 'giving PV of=', round(pv_for_imp_vol_est, decimals))

Estimated implied volatility for pv_target_val= 12 is 0.19227
Compare with BS price for implied vol of = 0.19227 giving PV of= 12.0
